In [1]:
import requests
import time
from pymongo import MongoClient
from tqdm import tqdm
from dotenv import load_dotenv
import os
from requests.exceptions import ConnectionError, Timeout, HTTPError
from http.client import RemoteDisconnected

# === Load environment variables ===
load_dotenv(dotenv_path=r"C:\Users\madhu\OneDrive\Desktop\Project - Git Hub Recommendation\Git-Hub-Recommendation\password.env")
GITHUB_TOKEN = os.getenv("GITHUB_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")

if not GITHUB_TOKEN:
    raise ValueError("❌ GITHUB_API_KEY not found in environment variables.")
if not MONGO_URI:
    raise ValueError("❌ MONGO_URI not found in environment variables.")

# === Setup GitHub headers ===
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}
user_data
# === MongoDB setup ===
client = MongoClient(MONGO_URI)
db = client["github_db"]
collection = db[""]

# === Helper Functions ===
def get_json(url, retries=3, backoff=5):
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code == 403:  # rate limit
                print("⚠️ Rate limit reached. Sleeping for 60 seconds.")
                time.sleep(60)
                continue
            response.raise_for_status()
            return response.json()
        except (ConnectionError, RemoteDisconnected, Timeout) as e:
            print(f"⚠️ Connection error on attempt {attempt+1}/{retries}: {e}")
            time.sleep(backoff)
        except HTTPError as e:
            print(f"⚠️ HTTP error: {e}")
            break
    raise Exception(f"❌ Failed to fetch {url} after {retries} attempts.")

def get_users_batch(since=0, per_page=100):
    url = f"https://api.github.com/users?since={since}&per_page={per_page}"
    return get_json(url)

def get_user_details(username):
    url = f"https://api.github.com/users/{username}"
    return get_json(url)

def get_list(url, key='login'):
    return [item[key] for item in get_json(url)]

def get_starred_or_subs(url):
    return [item['html_url'] for item in get_json(url)]

def get_languages(username):
    repos = get_json(f"https://api.github.com/users/{username}/repos")
    language_data = {}
    for repo in repos:
        langs = get_json(repo['languages_url'])
        for lang, count in langs.items():
            language_data[lang] = language_data.get(lang, 0) + count
    return language_data

def get_total_commits(username):
    repos = get_json(f"https://api.github.com/users/{username}/repos")
    total = 0
    for repo in repos:
        if repo.get('fork'):
            continue
        url = repo['commits_url'].replace("{/sha}", "")
        commits = requests.get(url, headers=HEADERS, params={'per_page': 1}).json()
        if isinstance(commits, list):
            total += len(commits)
    return total

# === Core Function to Fetch and Store User Data ===
def collect_and_store_user(username):
    try:
        user = get_user_details(username)
        data = {
            "Login": username,
            "Name": user.get("name"),
            "Bio": user.get("bio"),
            "Public Repositories": user.get("public_repos"),
            "Followers Count": user.get("followers"),
            "Following Count": user.get("following"),
            "Created At": user.get("created_at"),
            "Updated At": user.get("updated_at"),
            "Avatar URL": user.get("avatar_url"),
            "Profile URL": user.get("html_url"),
            "Followers List": get_list(f"https://api.github.com/users/{username}/followers"),
            "Following List": get_list(f"https://api.github.com/users/{username}/following"),
            "Starred Repositories": get_starred_or_subs(f"https://api.github.com/users/{username}/starred"),
            "Subscriptions": get_starred_or_subs(f"https://api.github.com/users/{username}/subscriptions"),
            "Organizations": get_list(f"https://api.github.com/users/{username}/orgs"),
            "Languages": get_languages(username),
            "Total Commits": get_total_commits(username)
        }
        collection.update_one({"Login": username}, {"$set": data}, upsert=True)
        print(f"✅ Data stored for {username}")
    except Exception as e:
        print(f"❌ Error for {username}: {e}")

# === Batch Processing ===
def batch_process_users(start_since=0, batches=1, per_page=10):
    since = start_since
    for _ in range(batches):
        users = get_users_batch(since, per_page)
        if not users:
            break
        for user in tqdm(users, desc="Processing batch"):
            collect_and_store_user(user['login'])
        since = users[-1]['id']

# === Example Usage ===
# Process first 2 batches of 100 users each
batch_process_users(start_since=0, batches=2, per_page=25)


Processing batch:   4%|▍         | 1/25 [00:30<12:14, 30.59s/it]

✅ Data stored for mojombo


Processing batch:   8%|▊         | 2/25 [01:08<13:24, 34.98s/it]

✅ Data stored for defunkt


Processing batch:  12%|█▏        | 3/25 [01:23<09:24, 25.65s/it]

...

✅ Data stored for indirect


Processing batch:  96%|█████████▌| 24/25 [10:41<00:27, 27.86s/it]

✅ Data stored for fearoffish


Processing batch: 100%|██████████| 25/25 [11:16<00:00, 27.04s/it]

✅ Data stored for ry
